In [7]:
import pickle
import numpy as np
from collections import defaultdict

# --- USER SETTING ---
PERCENTILE = 75  # flag all windows above this percentile (e.g. 95th percentile)

# --- Load Results ---
with open('05_20_42.pkl', 'rb') as f:
    all_results = pickle.load(f)

# --- Compute Anomaly Scores ---
anomaly_scores = defaultdict(lambda: {'score_sum': 0.0, 'count': 0})

for iteration_result in all_results:
    buckets = iteration_result['buckets']
    bucket_results = iteration_result['bucket_results']

    for bucket_result in bucket_results:
        bucket_idx = bucket_result['bucket_idx']
        final_results = bucket_result['final_results']
        indices_in_bucket = buckets[bucket_idx]

        mean = np.mean(final_results)
        std = np.std(final_results) if np.std(final_results) != 0 else 1e-8

        for i, idx in enumerate(indices_in_bucket):
            sim = final_results[i]
            deviation = abs(sim - mean) / std
            anomaly_scores[idx]['score_sum'] += deviation
            anomaly_scores[idx]['count'] += 1

# --- Final Score Calculation ---
final_scores = {
    idx: score_data['score_sum'] / score_data['count']
    for idx, score_data in anomaly_scores.items()
}

# --- Compute Threshold by Percentile ---
all_score_values = list(final_scores.values())
threshold = np.percentile(all_score_values, PERCENTILE)

# --- Flag Anomalous Windows ---
anomalous_windows = {
    idx: score for idx, score in final_scores.items() if score >= threshold
}

print(f"\n{len(anomalous_windows)} anomalous windows detected above the {PERCENTILE}th percentile (threshold ≈ {threshold:.4f}):\n")
for idx, score in sorted(anomalous_windows.items()):
    print(f"Window {idx} - Score: {score:.4f}")

# --- Save to file ---
with open("flagged_anomalous_windows.txt", "w") as f:
    for idx, score in sorted(anomalous_windows.items()):
        f.write(f"Window {idx} - Score: {score:.4f}\n")

print("\nSaved to 'flagged_anomalous_windows.txt'")



40 anomalous windows detected above the 75th percentile (threshold ≈ 0.8726):

Window 7 - Score: 0.9155
Window 8 - Score: 0.8925
Window 9 - Score: 0.8788
Window 11 - Score: 0.8730
Window 21 - Score: 0.8737
Window 47 - Score: 0.8744
Window 89 - Score: 0.8756
Window 107 - Score: 0.8847
Window 108 - Score: 0.9191
Window 109 - Score: 0.9168
Window 110 - Score: 0.9151
Window 111 - Score: 0.9292
Window 112 - Score: 0.9253
Window 113 - Score: 0.9352
Window 114 - Score: 0.9498
Window 116 - Score: 0.8799
Window 117 - Score: 0.9416
Window 118 - Score: 0.9259
Window 119 - Score: 0.8789
Window 121 - Score: 0.9451
Window 123 - Score: 0.9569
Window 124 - Score: 0.9123
Window 125 - Score: 0.8841
Window 126 - Score: 0.8999
Window 127 - Score: 0.8813
Window 128 - Score: 0.9142
Window 129 - Score: 0.8935
Window 130 - Score: 0.8884
Window 131 - Score: 0.9214
Window 132 - Score: 0.8789
Window 133 - Score: 0.9354
Window 134 - Score: 0.9418
Window 135 - Score: 0.9359
Window 136 - Score: 0.8847
Window 137 -